In [1]:
import sys
sys.path.append('../src')

In [3]:


import os
import numpy as np
import pcalg
import networkx as nx
from conditional_independence import partial_correlation_suffstat, partial_correlation_test
from mcmc.mcmc import PartitionMCMC
from mcmc.data import SyntheticDataset
from mcmc.proposals import PartitionProposal
from mcmc.scores import BGeScore
from mcmc.utils.graph_utils import generate_DAG, generate_key_from_adj_matrix, indep_test_func, cpdag_to_dag
from mcmc.utils.partition_utils import build_partition
from mcmc.evaluation.metrics import kl_divergence, jensen_shannon_divergence, mean_squared_error, mean_absolute_error
from mcmc.inference.posterior import generate_all_dags_keys, generate_all_dags, compute_true_distribution
from multiprocess import Pool

NUM_NODES = [3,5]
NUM_OBSERVATIONS = 200
N_ITERATIONS = 30000
N_REPEATS = 100
EDGE_P = [0.25, 0.5]

np.random.seed(32)

KLD = {}
JSD = {}
MSE = {}
MAE = {}

def f(args):
    num_nodes, edge_p = args
    for n_repeat in range(N_REPEATS):
        dag = generate_DAG(num_nodes, edge_p, np.random.randint(0, 100))
        node_labels = [chr(ord('a') + i) for i in range(num_nodes)]
        synthetic_data = SyntheticDataset(num_nodes=num_nodes, num_obs=NUM_OBSERVATIONS, node_labels=node_labels, true_dag=dag, degree=num_nodes-1)

        (g, sep_set) = pcalg.estimate_skeleton(indep_test_func=indep_test_func, data_matrix=synthetic_data.data.values, alpha=0.01)
        g = pcalg.estimate_cpdag(skel_graph=g, sep_set=sep_set)
        initial_graph = nx.to_numpy_array(g)
        initial_graph = cpdag_to_dag(initial_graph)

        score = BGeScore(synthetic_data.data, initial_graph)
        partition = build_partition(incidence=initial_graph, node_labels=list(synthetic_data.data.columns))
        proposal = PartitionProposal(partition)

        M = PartitionMCMC(init_part=partition, max_iter=N_ITERATIONS, proposal_object=proposal, score_object=score)

        mcmc_results, acceptance = M.run()
        graphs = M.get_mcmc_res_graphs(mcmc_results)
        BURN_IN = 0.1
        key = generate_key_from_adj_matrix(synthetic_data.adj_mat.values)
        keys, counts = np.unique([generate_key_from_adj_matrix(g) for g in graphs[int(BURN_IN*len(graphs)):]], return_counts=True)

        all_dags = generate_all_dags(data=synthetic_data.data, my_score=BGeScore)
        true_distribution = compute_true_distribution(all_dags)
        dst = {key:count/sum(counts) for key, count in zip(keys, counts)}
        approx_distribution = {key: (dst[key] if key in dst else 0) for key in true_distribution.keys()}

        KLD[num_nodes][edge_p].append(kl_divergence(approx_distribution, true_distribution))
        JSD[num_nodes][edge_p].append(jensen_shannon_divergence(approx_distribution, true_distribution))
        MSE[num_nodes][edge_p].append(mean_squared_error(approx_distribution, true_distribution))
        MAE[num_nodes][edge_p].append(mean_absolute_error(approx_distribution, true_distribution))

        np.savez(f'results/ablation_nodes={num_nodes}_edgep={edge_p}_r={n_repeat}.npz', dag=dag, init_dag=initial_graph, data=synthetic_data.data, node_labels=node_labels, posterior=approx_distribution, true_posterior=true_distribution, kld=KLD[num_nodes][edge_p][-1], jsd=JSD[num_nodes][edge_p][-1], mse=MSE[num_nodes][edge_p][-1], mae=MAE[num_nodes][edge_p][-1])
    print(num_nodes, edge_p, np.mean(KLD[num_nodes][edge_p]), np.std(KLD[num_nodes][edge_p]), np.mean(MSE[num_nodes][edge_p]), np.std(MSE[num_nodes][edge_p]))


In [4]:

os.makedirs('results', exist_ok=True)
processes = []
for num_nodes in NUM_NODES:
    KLD[num_nodes] = {}
    JSD[num_nodes] = {}
    MSE[num_nodes] = {}
    MAE[num_nodes] = {}
    for edge_p in EDGE_P:
        KLD[num_nodes][edge_p] = []
        JSD[num_nodes][edge_p] = []
        MSE[num_nodes][edge_p] = []
        MAE[num_nodes][edge_p] = []

with Pool(4) as p:
    p.map(f, [(num_nodes, edge_p) for num_node in NUM_NODES for edge_p in EDGE_P])

Initialise: Initialise:   0.000000.00000

Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise:  0.00000
Total 5 node DAGs generated = 29281
Initialise

AssertionError: 

In [ ]:
from matplotlib import pyplot as plt

np.savez(f'results/summary.npz', kld=KLD, jsd=JSD, mse=MSE, mae=MAE)